In [1]:
pip install mysqlclient

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
'define all the operations with DB connection using MySQLdb package'
import MySQLdb

class Query:
	""" 
	The Query class abstracts all operations with the database
	"""

	def __init__(self, db, table):
		"""
		constructor, stores inside connection and table
			db: created database connection
			table: table name
		"""
		self.db = db
		self.cursor = db.cursor()
		self.table = table

	def execute(self, sql):
		"""
		executes the given query
			sql: query, as a string
		"""
		try:
			self.cursor.execute(sql)
			self.db.commit()		
		except:
			self.db.rollback()
			print ("execution of '%s' failed" % sql)
			raise

	def drop(self):
		'drops table'
		sql = "DROP TABLE IF EXISTS " + self.table
		self.execute(sql)

	def create(self, attrs):
		"""
		creates table with specified attributes
			attrs: the attributes
		"""
		sql = "CREATE TABLE " + self.table + " (" + attrs + ")"
		self.execute(sql)

	def insert(self, attrs, val):
		sql = "INSERT INTO " + self.table + " (" + attrs + ") VALUES (" + val + ")"
		self.execute(sql)

	def insertlist(self, attrs, vals):
		try:
			for val in vals:
				sql = "INSERT INTO " + self.table + " (" + attrs + ") VALUES (" + val + ")"
				self.cursor.execute(sql)
			db.commit()
		except:
			db.rollback()


In [13]:
import mysql.connector

'open database connection'
db = mysql.connector.connect(host="localhost",user="root",password="1q2w3e",database="userdb")

'create Elev table'
e = Query(db, "Elev")
e.drop()
attrs = "eId int, nume text"
e.create(attrs)

'insert rows'
vals = []
attrs = "eId, nume"
vals = []
vals += ["1, 'Inoel'"]
vals += ["2, 'Diana'"]
vals += ["3, 'Vasile'"]
vals += ["4, 'Costel'"]
vals += ["5, 'George'"]
vals += ["6, 'Anca'"]
e.insertlist(attrs, vals)

'create Profesor table'
p = Query(db, "Profesor")
p.drop()
attrs = "pId int, nume text, materie text"
p.create(attrs)


'insert rows'
vals = []
attrs = "pId, nume, materie"
vals = []
vals += ["1, 'Popescu', 'geografie'"]
vals += ["2, 'Manoliu', 'matematica'"]
vals += ["3, 'Petrescu', 'romana'"]
p.insertlist(attrs, vals)


'create Catalog table'
c = Query(db, "Catalog")
c.drop()
attrs = "eId int, pId int, data date, nota int"
c.create(attrs)


'insert rows'
vals = []
attrs = "eId, pId, data, nota"
vals = []
vals += ["1, 1, '2022-04-20', 7"]
vals += ["2, 1, '2022-04-20', 9"]
vals += ["3, 2, '2022-04-20', 4"]
vals += ["2, 2, '2022-04-20', 10"]
vals += ["4, 2, '2022-04-20', 8"]
vals += ["1, 3, '2022-04-21', 8"]
vals += ["2, 3, '2022-04-21', 8"]
vals += ["3, 3, '2022-04-21', 8"]
vals += ["4, 3, '2022-04-21', 8"]
vals += ["6, 3, '2022-04-21', 8"]
vals += ["1, 2, '2022-04-22', 8"]
vals += ["1, 1, '2022-04-22', 8"]
vals += ["2, 2, '2022-04-22', 10"]
vals += ["2, 1, '2022-04-22', 10"]
vals += ["6, 2, '2022-04-22', 9"]
vals += ["6, 1, '2022-04-22', 9"]

c.insertlist(attrs, vals)

In [3]:
def execute_query(query):
    cursor = db.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    for row in result:
        print(row)

In [4]:
execute_query("SELECT * FROM Elev")

(1, 'Inoel')
(2, 'Diana')
(3, 'Vasile')
(4, 'Costel')
(5, 'George')
(6, 'Anca')


In [5]:
execute_query("SELECT * FROM Profesor")

(1, 'Popescu', 'geografie')
(2, 'Manoliu', 'matematica')
(3, 'Petrescu', 'romana')


In [6]:
execute_query("select elev.nume, materie, data, nota from elev, catalog, profesor \
                where elev.eid=catalog.eid and catalog.pId=profesor.pid")

('Inoel', 'geografie', datetime.date(2022, 4, 22), 8)
('Inoel', 'matematica', datetime.date(2022, 4, 22), 8)
('Inoel', 'romana', datetime.date(2022, 4, 21), 8)
('Inoel', 'geografie', datetime.date(2022, 4, 20), 7)
('Diana', 'geografie', datetime.date(2022, 4, 22), 10)
('Diana', 'matematica', datetime.date(2022, 4, 22), 10)
('Diana', 'romana', datetime.date(2022, 4, 21), 8)
('Diana', 'matematica', datetime.date(2022, 4, 20), 10)
('Diana', 'geografie', datetime.date(2022, 4, 20), 9)
('Vasile', 'romana', datetime.date(2022, 4, 21), 8)
('Vasile', 'matematica', datetime.date(2022, 4, 20), 4)
('Costel', 'romana', datetime.date(2022, 4, 21), 8)
('Costel', 'matematica', datetime.date(2022, 4, 20), 8)
('Anca', 'geografie', datetime.date(2022, 4, 22), 9)
('Anca', 'matematica', datetime.date(2022, 4, 22), 9)
('Anca', 'romana', datetime.date(2022, 4, 21), 8)


<h4>Problema 5</h4>
Gasiti elevii care au mai mult de o singura nota si calculati-le mediile
la acele materii (la care au mai mult de o singura nota). Afisati
numele elevului, materia si media acestuia. Ordonati dupa numele
elevului apoi dupa materie.

In [7]:
query="select elev.nume, materie, cast(avg(c1.nota) as decimal(10,2)) \
        from catalog as c1, catalog as c2, elev, profesor \
        where c1.eid=elev.eid and profesor.pId=c1.pId \
        and c2.eid=elev.eid and profesor.pId=c2.pId \
        and c1.data<>c2.data \
        group by elev.nume, c1.pid \
        order by elev.nume, c1.pid;"
execute_query(query)

('Diana', 'geografie', Decimal('9.50'))
('Diana', 'matematica', Decimal('10.00'))
('Inoel', 'geografie', Decimal('7.50'))


<h4>Problema 6</h4>
Calculati mediile pentru toti elevii, pentru fiecare materie in parte.
Daca un elev nu are nota, media sa va fi 0. Afisati numele elevului,
materia si media sa. Ordonati dupa numele elevului apoi dupa
materie.


In [15]:
query="select elev.nume, materie, 0 as media from\
       elev, profesor\
       where (elev.nume, materie) not in\
       (select elev.nume, materie\
       from  catalog, elev,  profesor\
       where catalog.eID = elev.eID \
       and catalog.pID = profesor.pID\
       group by catalog.pID, catalog.eId)\
       union\
       (select elev.nume, materie, cast(avg(nota) as decimal(10,2)) as media\
       from  catalog, elev, profesor\
       where catalog.eID = elev.eID \
       and catalog.pID = profesor.pID\
       group by catalog.pID, catalog.eId)\
       order by nume,materie;" 
execute_query(query)

('Anca', 'geografie', Decimal('9.00'))
('Anca', 'matematica', Decimal('9.00'))
('Anca', 'romana', Decimal('8.00'))
('Costel', 'geografie', Decimal('0.00'))
('Costel', 'matematica', Decimal('8.00'))
('Costel', 'romana', Decimal('8.00'))
('Diana', 'geografie', Decimal('9.50'))
('Diana', 'matematica', Decimal('10.00'))
('Diana', 'romana', Decimal('8.00'))
('George', 'geografie', Decimal('0.00'))
('George', 'matematica', Decimal('0.00'))
('George', 'romana', Decimal('0.00'))
('Inoel', 'geografie', Decimal('7.50'))
('Inoel', 'matematica', Decimal('8.00'))
('Inoel', 'romana', Decimal('8.00'))
('Vasile', 'geografie', Decimal('0.00'))
('Vasile', 'matematica', Decimal('4.00'))
('Vasile', 'romana', Decimal('8.00'))


<h4>Problema 7</h4>
Calculati media generala a fiecarui elev. Atentie la elevii care nu au
note la anumite materii, acele materii nu intra la nota. Daca un elev
nu are nici o nota, media sa generala va fi 0. Afisati numele elevului
si media, ordonat dupa nume.

In [14]:
query="select nume, cast(avg(media) as decimal(10,2)) as media_generala from\
       (select elev.nume, materie, avg(nota) as media\
       from  catalog, elev, profesor\
       where catalog.eID = elev.eID \
       and catalog.pID = profesor.pID\
       group by catalog.pID, catalog.eId) m\
       group by nume\
       union\
       select nume, 0 from elev where eid not in (select eid from catalog)\
       order by nume"
execute_query(query)

('Anca', Decimal('8.67'))
('Costel', Decimal('8.00'))
('Diana', Decimal('9.17'))
('George', Decimal('0.00'))
('Inoel', Decimal('7.83'))
('Vasile', Decimal('6.00'))


<h4>Problema 8</h4>
Calculati media generala pentru acea scoala, incluzand si elevii care
au media generala 0.

In [10]:
query="select cast(avg(media_generala) as decimal(10,2)) as medie \
      from\
      (select nume, avg(media) as media_generala from\
      (select elev.nume, materie, avg(nota) as media\
      from  catalog, elev, profesor\
      where catalog.eID = elev.eID \
      and catalog.pID = profesor.pID\
      group by catalog.pID, catalog.eId) m\
      group by nume\
      union\
      select nume, 0 from elev where eid not in (select eid from catalog)\
      order by nume) h"
execute_query(query)

(Decimal('6.61'),)


In [11]:
'disconnect from the database'
db.close()